In [1]:
from transformers import AutoModel, AutoTokenizer
import os
import json

In [2]:
model = AutoModel.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def embed(tokens, i):
    assert isinstance(tokens[0], str) # batch size 1
    tokens = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
    output = model(**tokens).last_hidden_state[0]
    idx = [idx for idx, el in enumerate(tokens.word_ids()) if el == i]
    vector = output[idx].mean(dim=0)
    return vector 

In [4]:
import pandas as pd
df = pd.DataFrame([json.loads(el) for el in open("factuality_embed.jsonl")])

In [5]:
df.sample(2)

,vector,word,label,split,idx
11196,"[-0.3263489305973053, 0.12503716349601746, 0.0...",existing,2.25,train,2521
15391,"[0.23019175231456757, -0.22958876192569733, 0....",said,3.00,train,4241


In [6]:
train = df[df.split.isin(["train", "dev"])]
test = df[df.split.isin(["test"])]

In [7]:
import numpy as np

X_train = np.vstack(train.vector)
y_train = train.label

X_test = np.vstack(test.vector)
y_test = test.label

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X_train, y_train)

In [ ]:
#!pip instalhttp://localhost:8888/notebooks/Desktop/%D0%BA%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F%203%20%D0%BA%D1%83%D1%80%D1%81/corpora/baseline%20(2)-Copy1.ipynb#l deep_translator
#!python -m spacy download en_core_web_trf
import spacy_transformers 
import spacy
import en_core_web_trf

nlp = spacy.load("en_core_web_trf")
nlp = en_core_web_trf.load()

In [10]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='ru', target='en')
import numpy as np


def roots(sentence):
  id = 0
  roots = {}
  for token in nlp(sentence):
    if token.dep_ == 'ROOT':
      roots[token.text] = id 
    id += 1

    if roots:
      target_id = 0
      for root in roots:
        span = [roots[root], roots[root] + 1]
        target_id += 1
        return span


def translate(original):
  translated = translator.translate(original, dest='en', src='ru')
  return translated

def embed(tokens, i):
    assert isinstance(tokens[0], str) # batch size 1
    tokens = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
    output = model(**tokens).last_hidden_state[0]
    idx = [idx for idx, el in enumerate(tokens.word_ids()) if el == i]
    vector = output[idx].mean(dim=0)
    return vector 

def get_factuality(sentence):
    translated = translate(sentence)
    index = roots(translated)
    doc = nlp(translated)
    translated = [w.text for w in doc]
    print(translated, index)
    factuality_score = reg.predict([embed(translated, index[0]).detach().numpy()])[0]
    if factuality_score > 3:
        factuality_score = 3.00
    elif factuality_score < 0:
        factuality_score *= 1.5
    elif factuality_score < -1:
        factuality_score *= 2
    elif factuality_score < 2 and 'не' in sentence.split():
        factuality_score -= 3
    if factuality_score < -3:
        factuality_score = -3.00

    return round(factuality_score, 2)

In [12]:
import telebot

botTimeWeb = telebot.TeleBot('6235401363:AAGKW404MvQMf_cSGSc1DNqSGWBCbfCk_Ks')

from telebot import types

In [13]:
@botTimeWeb.message_handler(commands=['start'])
def startBot(message):
  first_mess = f"Привет! Я умею предсказывать значение фактуальности предложений в тексте на <b>русском и английском языках</b>.\n\nФактуальность события выражает степень уверенности говорящего в достоверности той информации, которую он сообщает. Выражается фактуальность при помощи вещественного числа от <b>-3</b> до <b>3</b>, от непроизодшего события до достоверно случившегося соотвественно. В предложениях с несколькими сказуемыми модель будет определять значение <b>только для первого</b>.\n\nПодробнее о фактуальности и модели можно прочитать по команде /help.\n\nВведи своё предложение, чтобы начать."
  markup = types.InlineKeyboardMarkup()
  button_yes = types.InlineKeyboardButton(text = 'Да', callback_data='yes')
  markup.add(button_yes)
  markup = types.InlineKeyboardMarkup()
  markup.add(types.InlineKeyboardButton("Cвязаться с автором.", url="https://t.me/eapocs"))
  botTimeWeb.send_message(message.chat.id, first_mess, parse_mode='html', reply_markup=markup)

In [14]:
@botTimeWeb.message_handler(commands=['help'])
def helpBot(message):
    first_mess = f"О чём я могу тебе рассказать?"
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton('Бот молчит')
    btn2 = types.KeyboardButton("Немного теории")
    btn3 = types.KeyboardButton("Подробнее о модели")
    markup.add(btn1, btn2, btn3)
    botTimeWeb.send_message(message.chat.id, first_mess, parse_mode='html', reply_markup=markup)

In [15]:
@botTimeWeb.callback_query_handler(func=lambda call:True)
def response(function_call):
  if function_call.message:
     if function_call.data == "yes":
        second_mess = "Введи своё предложение."
        #markup = types.InlineKeyboardMarkup()
        #markup.add(types.InlineKeyboardButton("Cвязаться с автором.", url="https://t.me/eapocs"))
        botTimeWeb.send_message(function_call.message.chat.id, second_mess, reply_markup=markup)
        botTimeWeb.answer_callback_query(function_call.id)

In [16]:
@botTimeWeb.message_handler(content_types=["text"])
def repeat_all_messages(message):
    if(message.text == "Бот молчит"):
        botTimeWeb.send_message(message.chat.id, parse_mode='html', text="Попробуй отправить предложение снова или введи другое предложение.\n\nБот может не отвечать на предложения, которые введены в некорректной форме. Предложения должны быть на русском или английском.")
    elif(message.text == "Немного теории"):
        botTimeWeb.send_message(message.chat.id, parse_mode='html', text="Фактуальность — степень уверенности говорящего в том, что некоторое событие произошло. Вершиной события в предложении является сказуемое — именно для него определяется значение фактуальности.\n\nРазличные степени фактуальности могут выражаться разными способами, например:\n— модальность (<i>Послышались всплески: <b>наверное</b>, это маленькая зелёная лягушка прыгает по лужам.</i>)\n— дополнительный источник информации, или эвиденциальность (<i>Послышались всплески: <b>мальчик сказал</b>, что это маленькая зелёная лягушка прыгает по лужам.</i>)\n— наклонение (условное наклонение (<i>бы</i>) и повелительное наклонение (<i>прыгай!</i>) получают отрицательное значение)\n\nПримеры:\n<i>Лягушка <b>прыгала</b> по лужам</i> (3.0)\n<i>Лягушка не <b>любит</b> прыгать по лужам</i> (-3.0)\n<i>Вероятно, она <b>прыгнула</b> в лужу</i> (2.0)\n<i>По их словам, лягушка <b>прыгает</b> по лужам без калош</i> (1.5)")
    elif(message.text == "Подробнее о модели"):
        botTimeWeb.send_message(message.chat.id, parse_mode='html', text="Эта модель была обучена на корпусе текстов объёмом более 10 тысяч предложений. Корпус был переведён с английского языка. Текст на входе токенизируется и выделяется вершина предложения при помощи библиотеки <i>spacy</i>. Каждое предложение получает векторное представление при помощи мультиязычного <i>BERT-а</i>, затем этот вектор передаётся модели линейной регрессии, которая предсказывает численное значение фактуальности.")
    else:
        factuality_score = get_factuality(message.text)
        print(message.text, message.from_user.username, factuality_score)
        botTimeWeb.send_message(message.chat.id, f'Значение фактуальности: {factuality_score}')

In [ ]:
botTimeWeb.infinity_polling()